# Apprentissage Automatique : BE 2 

## Premier apprentissage

In [2]:
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from core import *

In [3]:
np.random.seed(1) # pour que l'exécution soit déterministe

# N est le nombre de données d'entrée
# D_in est la dimension des données d'entrée
# D_h le nombre de neurones de la couche cachée
# D_out est la dimension de sortie (nombre de neurones de la couche de sortie

N, D_in, D_h, D_out = 30, 2, 10, 3
lr = 0.01 # learning rate

In [4]:
np.random.seed(1)
lLoss=[]

X = np.random.random((N, D_in))
Y = np.random.random((N, D_out))


'''Initialisation aléatoire des poids du réseau'''
W1,b1=declareLayer(D_in,D_h)
W2,b2=declareLayer(D_h,D_out)

''' Learning '''

for ii in tqdm(range(100000),desc='Learning...'):
    '''Forward '''
    O1 = forwardPass(X,W1,b1)
    O2 = forwardPass(O1,W2,b2)
    Y_pred = O2
    
    lLoss=printLoss(ii,lLoss,Y,Y_pred)

    ''' Backprog ''' 
    l2_error,l2_delta=backProgLastLayer(Y,Y_pred)
    l1_error,l1_delta=backProgHiddenLayer(l2_delta,W2,O1)

    W2,b2,O1=updateLayer(l2_delta,O1,W2,b2,lr,bFirstLayer=False)
    W1,b1,X=updateLayer(l1_delta,X,W1,b1,lr,bFirstLayer=True)
    
print('Done !')

Learning...: 100%|██████████| 100000/100000 [00:06<00:00, 16633.67it/s]

Done !


In [19]:
plt.figure(figsize=(12,8))
plt.plot(lLoss,[250*ii for ii in range (len(lLoss))])
plt.grid(True)
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.show()

## MNIST



### Loading data

In [20]:
from mnist import MNIST
mdata = MNIST('MNIST-data')
DataApp, LabelApp = mdata.load_training()
DataTest, LabelTest = mdata.load_testing()

 22%|██▏       | 1108/5000 [01:00<03:30, 18.45it/s]

In [21]:
DataApp=np.asarray(DataApp)
LabelApp=np.asarray(LabelApp)
DataTest=np.asarray(DataTest)
LabelTest=np.asarray(LabelTest)

In [22]:
DataApp.shape

(60000, 784)

In [23]:
def decoupage_donnees(dataApp,labelApp,ratio):
    nr=int(ratio*dataApp.shape[0])
    l=list(range(dataApp.shape[0]))
    np.random.shuffle(l)

    dataLearning=np.asarray([dataApp[l[ii]] for ii in range(nr)])
    labelLearning=np.asarray([labelApp[l[ii]] for ii in range(nr)])
    dataEval=np.asarray([dataApp[l[ii+nr]] for ii in range(dataApp.shape[0]-nr)])
    labelEval=np.asarray([labelApp[l[ii+nr]] for ii in range(dataApp.shape[0]-nr)])
    return dataLearning,labelLearning,dataEval,labelEval

In [24]:
dataLearning,labelLearning,dataEval,labelEval=decoupage_donnees(DataApp,
                                                                LabelApp,
                                                                0.98)

In [25]:
def makeGT(x):
    z=np.zeros(10)
    z[x]=1
    return z

In [26]:
labelEvalArray=np.asarray(list(map(lambda x : makeGT(x) ,labelEval.tolist())))

In [40]:
dTest=DataTest[0:300]
lTest=LabelTest[0:300]

In [28]:
lTestArray=np.asarray(list(map(lambda x : makeGT(x) ,labelEval.tolist())))

### Launching learning

#### Loss functions

In [29]:
def calcLoss(rawGT,guess):
    return np.square(rawGT -guess).sum() / 2

In [30]:
def calcSuccess(gt,guess):
    test= gt==list(map(lambda x :np.argmax(x),guess))
    rightGuesses=len(list(filter(lambda x: x,test)))/len(gt)
    return rightGuesses

#### Learning

In [41]:
bashDataEval=dataEval[0:300,:]
bashLabelEval=labelEval[0:300]
bashLabelEvalArray=labelEvalArray[0:300,:]

In [42]:
bashDataEval.shape

(300, 784)

In [44]:
# N est le nombre de données d'entrée
# D_in est la dimension des données d'entrée
# D_h le nombre de neurones de la couche cachée
# D_out est la dimension de sortie (nombre de neurones de la couche de sortie

N=bashDataEval.shape[0]
D_in=bashDataEval.shape[1]
D_h=100
D_out=10
lr = 0.001

In [48]:
N


300

In [49]:
np.random.seed(1)
lLearningLoss=[]
lLearningSuccess=[]
lEvalLoss=[]
lEvalSuccess=[]
lTestLoss=[]
lTestSuccess=[]

'''Initialisation aléatoire des poids du réseau'''
W1,b1=declareLayer(D_in,D_h)
W2,b2=declareLayer(D_h,D_out)

''' Learning '''

for ii in tqdm(range(5000)):
        '''Forward '''
        O1 = forwardPass(bashDataEval,W1,b1)
        O2 = forwardPass(O1,W2,b2)
        Y_pred = O2

        if not ii % 250 :
            lLearningLoss.append(calcLoss(bashLabelEvalArray,Y_pred))
            lLearningSuccess.append(calcSuccess(bashLabelEval,Y_pred))
            
            O1Eval = forwardPass(dTest,W1,b1)
            O2Eval = forwardPass(O1Eval,W2,b2)
            Y_predEval = O2Eval
#             lEvalLoss.append(calcLoss(lTestArray,Y_predEval))
            lEvalSuccess.append(calcSuccess(lTest,Y_predEval))

        ''' Backprog ''' 
        l2_error,l2_delta=backProgLastLayer(bashLabelEvalArray,Y_pred)
        l1_error,l1_delta=backProgHiddenLayer(l2_delta,W2,O1)
        

        W2,b2,O1=updateLayer(l2_delta,O1,W2,b2,lr,bFirstLayer=False)
        W1,b1,dataEval=updateLayer(l1_delta,dataEval,W1,b1,lr,bFirstLayer=True)

  0%|          | 0/5000 [00:00<?, ?it/s]/home/apopov/Documents/ECL/S9/apprentissageAuto/apprentissage_auto/core.py:7: RuntimeWarning: overflow encountered in exp
  O = 1/(1+np.exp(-I))



ValueError: shapes (784,1200) and (300,100) not aligned: 1200 (dim 1) != 300 (dim 0)

In [ ]:
absisse=[ 250 * ii for ii in range(len(lLearningLoss))]

plt.figure(figsize=(12,8))
plt.subplot(211)
plt.plot(absisse,lLearningLoss)
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(212)
plt.grid(True)
plt.plot(absisse,lLearningSuccess)
plt.xlabel('Iterations')
plt.ylabel('Success ratio')

In [ ]:
absisse=[ 250 * ii for ii in range(len(lEvalLoss))]

plt.figure(figsize=(12,8))
# plt.subplot(211)
# plt.plot(absisse,lEvalLoss)
# plt.xlabel('Iterations')
# plt.ylabel('Loss')
# plt.grid(True)

plt.subplot(212)
plt.grid(True)
plt.plot(lEvalSuccess)
plt.xlabel('Iterations')
plt.ylabel('Success ratio')